In [11]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
!pip install gdown

In [9]:
import gdown
from google.colab import drive
# Update with your video's specific ID
file_id = "1WGpTP4-JJvlJNZyo-5n32rT38QDk5M6j"
url = f"https://drive.google.com/uc?id={file_id}"

output = "park_video.mp4"
gdown.download(url, output, quiet=False)

SOURCE_VIDEO_PATH = "/content//drive/MyDrive/Pose_Estimation/park_video.mp4"
MODEL = "/content/drive/MyDrive/Pose_Estimation/saved_model.pkl"

Downloading...
From: https://drive.google.com/uc?id=1WGpTP4-JJvlJNZyo-5n32rT38QDk5M6j
To: /content/park_video.mp4
100%|██████████| 52.5M/52.5M [00:00<00:00, 121MB/s]


In [5]:
!pip install mediapipe==0.10.9 scikit-learn==1.2.2
!pip install opencv-python==4.7.0.72

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
imbala

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 15.3 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.11.0.86
    Uninstalling opencv-python-4.11.0.86:
      Successfully uninstalled opencv-python-4.11.0.86


In [10]:
import cv2
import mediapipe as mp
import json
import os
import pickle
import numpy as np
import pandas as pd
import sklearn

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Load the pre-trained model
with open(MODEL, 'rb') as f:
    model = pickle.load(f)

# Video input and output paths
video_path = SOURCE_VIDEO_PATH
output_json_path = "/content/pose_action_estimations.json"
output_frames_folder = "/content/pose_frames"
output_video_path = "/content/pose_action_output_video.mp4"

# Create output directories
if not os.path.exists(output_frames_folder):
    os.makedirs(output_frames_folder)

if not os.path.exists(video_path):
    raise FileNotFoundError(f"Video file {video_path} not found")

# Open video capture
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise ValueError("Error opening video file")

# Video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Initialize data structure for JSON
pose_action_data = {}

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB for processing
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = holistic.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Draw landmarks
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
        )

        # Extract pose keypoints
        pose = results.pose_landmarks.landmark
        pose_row = np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten()
        row = pose_row

        # Predict action
        X = pd.DataFrame([row])
        action_class = model.predict(X)[0]
        action_prob = model.predict_proba(X)[0]
        max_prob = round(action_prob[np.argmax(action_prob)], 2)

        # Store data
        keypoints = [{"landmark_id": idx, "x": float(x), "y": float(y), "z": float(z), "visibility": float(v)}
                    for idx, (x, y, z, v) in enumerate(zip(pose_row[::4], pose_row[1::4], pose_row[2::4], pose_row[3::4]))]
        pose_action_data[f"frame_{frame_count}"] = {
            "keypoints": keypoints,
            "action": action_class,
            "confidence": max_prob
        }

    # Save annotated frame
    frame_output_path = os.path.join(output_frames_folder, f"frame_{frame_count:06d}.jpg")
    cv2.imwrite(frame_output_path, image)
    out.write(image)

    frame_count += 1

# Release resources
cap.release()
out.release()

# Save to JSON
with open(output_json_path, 'w') as f:
    json.dump(pose_action_data, f, indent=4)

print(f"Pose and action estimations saved to {output_json_path}")
print(f"Frames with pose annotations saved to {output_frames_folder}")
print(f"Output video with pose annotations saved to {output_video_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Pose and action estimations saved to /content/pose_action_estimations.json
Frames with pose annotations saved to /content/pose_frames
Output video with pose annotations saved to /content/pose_action_output_video.mp4
